<a href="https://colab.research.google.com/github/gabcmartins/scripts_projeto_doc/blob/main/idxstats_meets_diamond_output_laske.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import pandas as pd
import glob

In [ ]:
arquivos= sorted(glob.glob ("/home/gabrielacastilho/idxstats/*.idxstats.txt")

In [ ]:
colunas = ["CDS", "lenght", "mapped_reads", "unmapped_reads"]

In [ ]:
teste= pd.read_csv ("/home/gabrielacastilho/idxstats/ERR9578157.idxstats.txt", sep="\t")

In [ ]:
for arquivo in arquivos:
    # Lê o arquivo sem cabeçalho e atribui os nomes das colunas
    df = pd.read_csv(arquivo, sep="\t", names=colunas, header=None)

    # Salva com cabeçalho incluso (pode sobrescrever ou salvar como novo)
    df.to_csv(arquivo, sep="\t", index=False)


In [ ]:
teste.head()


,CDS,length,mapped_reads,unmapped_reads,Normalized_values
0,k141_64904_1,321,3,0,0.347229
1,k141_55632_1,144,5,0,1.290052
2,k141_55632_2,102,0,0,0.000000
3,k141_111263_1,234,6,0,0.952654
4,k141_74176_1,78,0,0,0.000000


In [ ]:
df = pd.read_csv(arquivo, sep="\t", names=["CDS", "length", "mapped_reads", "unmapped_reads"], header=None, dtype=str)
print(df.dtypes)
print(df.head())


In [ ]:
df = pd.read_csv(arquivo, sep="\t", names=["CDS", "length", "mapped_reads", "unmapped_reads"], header=0)
df = df[df["CDS"] != "CDS"]  # Remove qualquer linha extra de cabeçalho
df[["length", "mapped_reads", "unmapped_reads"]] = df[["length", "mapped_reads", "unmapped_reads"]].astype(int)

print(df.dtypes)  # Agora todas as colunas devem estar corretas
print(df.head())


In [ ]:
resultados = []

for arquivo in arquivos:
    df = pd.read_csv(arquivo, sep="\t", names=["CDS", "length", "mapped_reads", "unmapped_reads"], header=0)
    df = df[df["CDS"] != "CDS"]  # Remove qualquer linha extra de cabeçalho
    df[["length", "mapped_reads", "unmapped_reads"]] = df[["length", "mapped_reads", "unmapped_reads"]].astype(int)

    total_mapped = df["mapped_reads"].sum()
    amostra = arquivo.split("/")[-1].replace(".idxstats.txt", "")
    resultados.append([amostra, total_mapped])

# Criar um DataFrame com os resultados
df_resultado = pd.DataFrame(resultados, columns=["Amostra", "Total_Mapped_Reads"])
df_resultado.to_csv("/home/gabrielacastilho/idxstats/total_mapped_reads.csv", index=False)

print(df_resultado)

In [ ]:
for arquivo in arquivos:
    df= pd.read_csv(arquivo, sep='\t')
    total_mapped = df["mapped_reads"].sum()
    df['Normalized_values']= (df["mapped_reads"] * 10**9) / (df["length"] * total_mapped)
    df.to_csv(arquivo, sep='\t', index=False)


In [ ]:
total=len(teste)
print(total)

In [ ]:
zero_mapped = (teste["mapped_reads"] == 0).sum()

print(f"Número de CDS sem reads alinhados: {zero_mapped}")

In [ ]:
proporção= zero_mapped/total*100
print(f"Proporção de CDS sem reads alinhados: {proporção:.2f}%")


In [ ]:
resultados = []

# Loop sobre os arquivos
for arquivo in arquivos:
    df = pd.read_csv(arquivo, sep='\t')
    total = len(df)
    zero_mapped = (df["mapped_reads"] == 0).sum()
    proporcao = zero_mapped / total * 100

    # Extrair o nome da amostra do caminho completo
    amostra = arquivo.split("/")[-1].replace(".idxstats.txt", "")  # Pega só o nome sem o caminho
    resultados.append([amostra, proporcao])

# Converter os resultados em um DataFrame
df_resultados = pd.DataFrame(resultados, columns=["Amostra", "Proporção_CDS_Sem_Reads"])

# Salvar os resultados em um arquivo CSV
df_resultados.to_csv("proporcao_cds_sem_reads.csv", index=False)

print(df_resultados)

In [ ]:
for arquivo in arquivos:
    df= pd.read_csv(arquivo, sep='\t')
    df.rename (columns={'Normalized_values': 'Abundância'}, inplace=True)
    df.to_csv(arquivo, sep='\t', index=False)

In [ ]:
import os

def carregar_idxstats(caminho):
    df = pd.read_csv(caminho, sep='\t')
    df = df[['CDS', 'Abundância']]
    return df

def carregar_diamond(caminho):
    df = pd.read_csv(caminho, sep='\t')
    df = df[['qseqid', 'sseqid']]
    df.rename(columns={'qseqid': 'CDS', 'sseqid': 'Proteina'}, inplace=True)
    return df

def unificar_dados(idxstats_dir, diamond_dir, saida):
    todas_amostras = []

    for arquivo in os.listdir(idxstats_dir):
        if arquivo.endswith(".idxstats.txt"):
            amostra = arquivo.split(".")[0]
            caminho_idx = os.path.join(idxstats_dir, arquivo)
            caminho_diamond = os.path.join(diamond_dir, f"{amostra}_matches_best_hits.tsv")

            if os.path.exists(caminho_diamond):
                df_idx = carregar_idxstats(caminho_idx)
                df_diamond = carregar_diamond(caminho_diamond)

                df_merged = df_diamond.merge(df_idx, on='CDS', how='left')
                df_merged['Amostra'] = amostra
                todas_amostras.append(df_merged)
            else:
                print(f"[Aviso] Não encontrou Diamond para {amostra}")

    df_final = pd.concat(todas_amostras, ignore_index=True)
    df_final.to_csv(saida, sep='\t', index=False)
    print(f"Dados unificados salvos em: {saida}")

# Caminhos das pastas
dir_idxstats = "/home/gabrielacastilho/idxstats/"
dir_diamond = "/home/gabrielacastilho/outputs_diamond_best_hits/"
arquivo_saida = "dados_unificados.tsv"

# Rodar unificação
unificar_dados(dir_idxstats, dir_diamond, arquivo_saida)


Dados unificados salvos em: dados_unificados.tsv


In [ ]:
dados_unificados= pd.read_csv('/home/gabrielacastilho/dados_unificados.tsv', sep="\t")

NameError: name 'pd' is not defined

In [ ]:
print(len(dados_unificados))

2596051


In [ ]:
import os
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Caminhos dos arquivos
arquivo_unificado = "/content/drive/MyDrive/Doutorado/dados_unificados.tsv"
arquivo_amostras = "/content/drive/MyDrive/Doutorado/amostras.csv"
arquivo_saida = "/content/drive/MyDrive/Doutorado/dados_unificados_com_grupo.tsv"

# Carregar dados unificados
df_unificado = pd.read_csv(arquivo_unificado, sep='\t')

# Carregar dados das amostras
df_amostras = pd.read_csv(arquivo_amostras)
df_amostras.rename(columns={'Run Accession': 'Amostra', 'AMYLOID': 'Grupo'}, inplace=True)

# Unir os dados
df_final = df_unificado.merge(df_amostras[['Amostra', 'Grupo']], on='Amostra', how='left')

# Salvar o novo arquivo
df_final.to_csv(arquivo_saida, sep='\t', index=False)
print(f"Dados salvos em: {arquivo_saida}")


Dados salvos em: /content/drive/MyDrive/Doutorado/dados_unificados_com_grupo.tsv


In [ ]:
import pandas as pd

tabela=pd.read_csv('/content/drive/MyDrive/Doutorado/dados_unificados_com_grupo.tsv', sep='\t')
tabela.head()


,CDS,Proteina,Abundância,Amostra,Grupo
0,k141_100096_5,PUL0576_1,1.693629,ERR9578260,Negative
1,k141_100132_3,GH2.AXB28859.1,2.949103,ERR9578260,Negative
2,k141_100140_5,PUL0134_16,2.681332,ERR9578260,Negative
3,k141_100159_2,PUL0254_24,1.114511,ERR9578260,Negative
4,k141_10015_2,PUL0232_6,0.621523,ERR9578260,Negative
